In [11]:
import cv2
import numpy as np
from scipy.spatial import distance as dist
import numpy as np
import cv2

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications import MobileNetV2
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

from tensorflow import keras
from tensorflow.keras import layers

from imutils import paths
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [2]:
# Init variables
INIT_LR = 1e-4
EPOCHS = 50
BS = 32

imagePaths = list(paths.list_images("Resources//data3"))
imagePaths = [imagePath.replace("\\","//",-1) for imagePath in imagePaths]
data = []
labels = []

# loop over the image paths
for imagePath in imagePaths:
    # Extract the class label from directory
    label = imagePath.split("//")[2]
    labels.append(label)
    
    # Convert image to grey scale and 48x48
    image = load_img(imagePath, color_mode = "grayscale", target_size=(48, 48, 1))
    image = img_to_array(image)
    image = preprocess_input(image)
    
    data.append(image)

# Convert the data and labels to np arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)    

# Perform one-hot encoding on labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [3]:
# Create the model
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

model.compile(loss="binary_crossentropy", optimizer=Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS),metrics=["accuracy"])
history = model.fit(trainX,trainY,batch_size=BS,epochs=EPOCHS,validation_data=(testX, testY))

model.save("Resources/Trained_Data/test2.h5")

print("trained")

trained


In [ ]:
predIdxs = model.predict(testX, batch_size=BS)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,target_names=lb.classes_))

In [ ]:
import matplotlib.pyplot as plt

# plot the training loss and accuracy
N = EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch No.")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Resources/Training_Graph")
print("Training Completed")

In [ ]:
# # Define data generators
# # train_dir = 'Resources/test'
# # val_dir = 'Resources/train'
# # num_train = 28709
# # num_val = 7178
# # batch_num = 64
# # num_epoch = 50

# train_dir = 'Resources/data'
# val_dir = 'Resources/data2'
# num_train = 5
# num_val = 5
# batch_size = 1
# num_epoch = 5

# train_datagen = ImageDataGenerator(rescale=1./255)
# val_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(48,48),
#         batch_size=batch_size,
#         color_mode="grayscale",
#         class_mode='categorical')

# validation_generator = val_datagen.flow_from_directory(
#         val_dir,
#         target_size=(48,48),
#         batch_size=batch_size,
#         color_mode="grayscale",
#         class_mode='categorical')

# model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])

# model_info = model.fit(train_generator, steps_per_epoch=num_train // batch_size, 
#                                  epochs=num_epoch,validation_data=validation_generator, validation_steps=num_val // batch_size)

In [4]:
faceCascade = cv2.CascadeClassifier("Resources/cascade/haarcascade_frontalface_default.xml")

model2 = tf.keras.models.clone_model(model)

model.load_weights('Resources/Trained_Data/test1.h5')
model2.load_weights('Resources/Trained_Data/test2.h5')

# dictionary which assigns each label an emotion (alphabetical order)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

In [12]:
img = cv2.imread("Resources/images/faces.jpg")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(img, 1.3, 5)

img2 = img.copy()

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
    roi_gray=gray[y:y+w,x:x+h]
    roi_gray=cv2.resize(roi_gray,(48,48))
    img_pixels = image.img_to_array(roi_gray)
    img_pixels = np.expand_dims(img_pixels, axis = 0)
    img_pixels /= 255
    
    prediction = model.predict(img_pixels)
    maxindex = int(np.argmax(prediction))
    cv2.putText(img, emotion_dict[maxindex], (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.rectangle(img2,(x,y),(x+w,y+h),(255,0,0),thickness=7)
    prediction2 = model2.predict(img_pixels)
    maxindex2 = int(np.argmax(prediction2))
    cv2.putText(img2, emotion_dict[maxindex2], (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    #prediction, idx, probability = learn.predict(Image(pil2tensor(roi_gray, np.float32).div_(225)))
    #cv2.putText(img, str(prediction), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (225, 255, 255), 2)

resized_img = cv2.resize(img, (1000, 700))
cv2.imshow('Facial emotion analysis ',resized_img)


resized_img2 = cv2.resize(img2, (1000, 700))
cv2.imshow('Facial emotion analysis2 ',resized_img2)

cv2.waitKey(0)

-1

In [ ]:
print("hi")